In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Make a CNN step-by-step using pytorch:

In [2]:
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
# from torch.nn import Module
from torch import optim
from torch import nn



### Step-1: We import the data and make it ready for training
So basically we will make a class which will, (a) download the data (b) Transfer to tesor (c) make mini-batches for the train and test

In [3]:
class CSVDataset(Dataset):
    def __init__(self, path):
        # use a dataframe to load the CSV file
        df = read_csv(path, header=None)
        
        # strore input (X) and output(y) tensor
        self.X = df.values[:,:-1]
        self.y = df.values[:,-1]
        
        # ensure input data is floats  
        self.X = self.X.astype('float32')
        
        # label encode target (check online for meaning) 
        # and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape(len(self.y),1)
        
    # To determine the length of row 
    def __len__(self):
        return len(self.X)
    
    # Call input and output row by index 
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])
    
    # get indexes for train and test rows
#     def get_splits(self, n_test=0.2):
#         # we calculate test and train sizes (return numbers only)
#         test_size = round(len(self.X)*n_test)
#         train_size = len(self.X) - test_size
        
#         # We split the dataset by randomly choosing the entire Dataset/Mini-batch
#         return random_split(self, [train_size, test_size])   
         
        

### Step-2: Make model
We make class which takes input and produce output


In [4]:
class MLP(nn.Module):
    # during initialization we define: for each layer, (a) the number of features 
    # (b) the weights (c) the type of activation
    def __init__(self, n_input, n_hidden1, n_hidden_2, n_out):
        super(MLP, self).__init__()
        ###########################
        # hidden layer 1
        self.hidden1 = nn.Linear(n_input, n_hidden1)
        nn.init.kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = nn.ReLU()
        
        # hidden layer 2
        self.hidden2 = nn.Linear( n_hidden1, n_hidden_2)
        nn.init.kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = nn.ReLU()
        
        # Output layer 
        self.output = nn.Linear(n_hidden_2, n_out)
        nn.init.xavier_uniform_(self.output.weight)
        self.actout = nn.Sigmoid()
        
     # Now do Forward propagation
    def forward(self, X):
        # hidden layer 1
        X = self.hidden1(X)
        X = self.act1(X)
        # hidden layer 2
        X = self.hidden2(X)
        X = self.act2(X)
        # output layer 
        X = self.output(X)
        X = self.actout(X)
        return X

Use the CSVDatatset class to make a function which return train and test set

In [5]:
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

In [6]:
# prepare the data
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))
# print(train_dl.dataset)

235 116


### Step-3: Train model


In [7]:
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
        if (epoch+1)%10 == 0:
            print("Loss for epch ", epoch+1, "is ", loss.detach().numpy())
            

In [8]:
def evaluate_model(train_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):        
        # Get output using model (defined as MLP above)
        yhat = model(inputs)            
        # retrieve numpy array and round it
        yhat = yhat.detach().numpy()
        yhat = yhat.reshape(len(yhat), 1)            
        yhat = yhat.round()
        # actual values provided by data
        actual = targets.numpy()
        actual = actual.reshape(len(actual), 1)
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(predictions, actuals)  

In [9]:
# make a class prediction for one row of data
def predict(row, model):
    # convert to torch
    row = torch.tensor([row])
    # prediction
    yhat = model(row)
    # retrive to numpy
    yhat = yhat.detach().numpy()
    return yhat

### Step-4: Run
In each step, we have made a class and made functions which uses the class to produce our predicted output. In the next few lines we only will call the functions (i.e., no need to call classes as functions have already done that job)


In [10]:
# define online path from where data can be downloaded
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/ionosphere.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))

235 116


In [11]:
# define the network
model = MLP(n_input=34, n_hidden1=10, n_hidden_2=8, n_out=1 )

In [12]:
# train model
train_model(train_dl, model)

Loss for epch  10 is  0.39087227
Loss for epch  20 is  0.17673416
Loss for epch  30 is  0.1317012
Loss for epch  40 is  0.012886247
Loss for epch  50 is  0.051994693
Loss for epch  60 is  0.018117266
Loss for epch  70 is  0.0064347233
Loss for epch  80 is  0.02582776
Loss for epch  90 is  0.022949709
Loss for epch  100 is  0.26801574


In [13]:
# evaluate the model
acc = evaluate_model(test_dl, model)
print(acc)

None


In [15]:
# make a single prediction (expect class=1)
row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
yhat = predict(row, model)
print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))

Predicted: 0.998 (class=1)


### Other practice codes:

In [14]:
# Test the above code whie writing
def test():
    torch.manual_seed(42)
    X = torch.randint(0,10, size = (4, 10))
    y = torch.randint(0,2, size = (1, 10))
    return X, y, len(X)
X, y = test()
print(X,y)

ValueError: too many values to unpack (expected 2)

In [ ]:
    torch.manual_seed(42)
    a = torch.randint(0,10, size = (2,3,4))
    a, a[:,:-1], a[:,-1], a[1]
    